# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [13]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [14]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [15]:
# Example usage:
limit = 5
generator = square_root_generator(limit)
answer1 = 0
for sqrt_value in generator:
    answer1 += sqrt_value

#question 2 
limit = 13
generator = square_root_generator(limit)
for sqrt_value in generator:
    answer2 = sqrt_value



In [16]:
## Question 3 

import dlt



# define the connection to load to. 
# We now use duckdb, but you can switch to Bigquery later
Q3pipeline = dlt.pipeline(pipeline_name="hw_data",
						destination='duckdb', 
						dataset_name='hw3')

# run the pipeline with default settings, and capture the outcome
# pass the generator (people_1) as  data
info = Q3pipeline.run(people_1(),
                      table_name="users",
					  write_disposition="replace")

info = Q3pipeline.run(people_2(),
                      table_name="users",
					  write_disposition="append")
# show the outcome
print(info)


Pipeline hw_data load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw3
The duckdb destination used duckdb:////workspaces/Zoomcamp2024/03-dlt workshop/hw_data.duckdb location to store data
Load package 1707310433.6515117 is LOADED and contains no failed jobs


In [17]:
import duckdb
conn = duckdb.connect(f"{Q3pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{Q3pipeline.dataset_name}'")

In [18]:
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n users table below:")

users = conn.sql("SELECT * FROM users").df()
display(users)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 users table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707310432.9939706,2esYYCp46OYqBg,None
1,2,Person_2,27,City_A,1707310432.9939706,iVTO4hUEnyu/mQ,None
2,3,Person_3,28,City_A,1707310432.9939706,8ojXgV+yB2CSOA,None
3,4,Person_4,29,City_A,1707310432.9939706,2/bPjPFp+y0J1w,None
4,5,Person_5,30,City_A,1707310432.9939706,uVhlzoc+hGrIDw,None
5,3,Person_3,33,City_B,1707310433.6515117,dUZBddgdb0uwAQ,Job_3
6,4,Person_4,34,City_B,1707310433.6515117,xERi1qS8iZFlSA,Job_4
7,5,Person_5,35,City_B,1707310433.6515117,NQCwpyKlrEV2ZQ,Job_5
8,6,Person_6,36,City_B,1707310433.6515117,ONQs3Dm3mLl1Ew,Job_6
9,7,Person_7,37,City_B,1707310433.6515117,wONtj48mJRmMtg,Job_7


In [19]:
answer3 = conn.sql("SELECT sum(age) as 'answer' FROM users").df()
#display(answer)


In [20]:
#Question 4

import dlt



# define the connection to load to. 
# We now use duckdb, but you can switch to Bigquery later
Q4pipeline = dlt.pipeline(pipeline_name="hw_data",
						destination='duckdb', 
						dataset_name='hw3')

# run the pipeline with default settings, and capture the outcome
# pass the generator (people_1) as  data
info = Q4pipeline.run(people_1(),
                      table_name="users",
					  write_disposition="replace")

info = Q4pipeline.run(people_2(),
                      table_name="users",
					  write_disposition="merge",
                      primary_key = "id")
# show the outcome
print(info)



Pipeline hw_data load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw3
The duckdb destination used duckdb:////workspaces/Zoomcamp2024/03-dlt workshop/hw_data.duckdb location to store data
Load package 1707310435.061645 is LOADED and contains no failed jobs


In [21]:
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n users table below:")

users = conn.sql("SELECT * FROM users").df()
display(users)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 users table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707310434.4381218,kOMhtfqfXZ6uiA,None
1,2,Person_2,27,City_A,1707310434.4381218,vaU5Qx/qeXWgvw,None
2,8,Person_8,38,City_B,1707310435.061645,LCPP3S5gL9fH8g,Job_8
3,5,Person_5,35,City_B,1707310435.061645,DryhxWPC/tmW3w,Job_5
4,7,Person_7,37,City_B,1707310435.061645,cw8DgtoE20JO4w,Job_7
5,4,Person_4,34,City_B,1707310435.061645,EkjMCLkYk/MvTw,Job_4
6,3,Person_3,33,City_B,1707310435.061645,9qOyX9PCmDWlww,Job_3
7,6,Person_6,36,City_B,1707310435.061645,XV4HS4vSME+OXg,Job_6


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [22]:
answer4 = conn.sql("SELECT sum(age) as 'answer' FROM users").df()
#display(answer)


In [23]:
print('Answer for Q1 :', answer1)
print('Answer for Q2 :', answer2)  
print('Answer for Q3 :', answer3['answer'].values[0])  
print('Answer for Q4 :', answer4['answer'].values[0])  


Answer for Q1 : 8.382332347441762
Answer for Q2 : 3.605551275463989
Answer for Q3 : 353.0
Answer for Q4 : 266.0
